<a href="https://colab.research.google.com/github/aliciachenw/11785-project/blob/main/load_labeled_part.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# **split the training set into labeled and unlabeled**



In [1]:
# load google drive
from google.colab import drive
drive.mount('/content/gdrive') 

Mounted at /content/gdrive


In [2]:
import os, json, random

def split_labeled_unlabeled(img_dir, split=None):
  """
  randomly divide the data in img_dir into labeledand unlabeled
  :param: img_dir: directory saving the 
  :param: split: ratio of labeled data to all data
  """
  json_file = os.path.join(img_dir, "thermal_annotations.json")
  with open(json_file) as f:
    json_data = json.load(f)
    imgs_info = json_data["info"]
    imgs_categories = json_data["categories"]
    imgs_license = json_data["licenses"]
    imgs_anns = json_data["annotations"]
    imgs_infos = json_data["images"]
  
  dataset_dicts = []
  pointer = 0

  for idx, v in enumerate(imgs_infos):
    record = {}
    record["image_id"] = idx
    record["images"] = v
    objs = []
    while pointer < len(imgs_anns):
      annotation = imgs_anns[pointer]
      if annotation["image_id"] == record["image_id"]:
        if annotation["category_id"] == 1:
          obj = annotation
          objs.append(obj)
        pointer += 1
      else:
        record["annotations"] = objs
        dataset_dicts.append(record)
        break
  
  if split != None:
    length_data = len(dataset_dicts)
    random_train = int(split * length_data)
    random.shuffle(dataset_dicts)
    train_dicts = dataset_dicts[:random_train]
    vld_dicts = dataset_dicts[random_train:]

    with open(img_dir + '/labeled.json', 'w') as fp:
      training_images = []
      training_annotations = []
      for instance in train_dicts:
        training_images.append(instance["images"])
        training_annotations = training_annotations + instance["annotations"]

      training_dicts = {}
      training_dicts["info"] = imgs_info
      training_dicts["categories"] = imgs_categories
      training_dicts["license"] = imgs_license
      training_dicts["annotations"] = training_annotations
      training_dicts["images"] = training_images
      json.dump(training_dicts, fp, indent=4)


In [3]:
dataset_dir = "/content/gdrive/My Drive/FLIR/"
split_labeled_unlabeled(dataset_dir + "/train", 0.2)